------
#### Preliminary steps

**Note:** For simplicity, we will just use default task and network instanciations. 

In [ ]:
import numpy as np
from fna.tasks.symbolic import SymbolicSequencer
from fna.tasks.symbolic.embeddings import VectorEmbeddings
from fna.encoders import NESTEncoder, InputMapper
from examples import example_defaults

* Initialize NEST kernel

In [2]:
resolution = 0.1
example_defaults.reset_kernel(resolution=resolution)

* Create target network (SNN) - here load a simple default SNN

In [3]:
snn, snn_recurrent_connections = example_defaults.default_network(N=100, record_spikes=True)

[__init__.py:47 - INFO] Initializing Brunel BRN architecture (NEST-simulated)
[__init__.py:97 - INFO] Creating populations:
[__init__.py:121 - INFO] - Population E, with ids [1-80]
[__init__.py:121 - INFO] - Population I, with ids [81-100]
[__init__.py:417 - INFO] Connecting Devices: 
[__init__.py:129 - INFO]   - Attaching spike_detector with gid [(101,)] to population E
[__init__.py:438 - INFO] - Connecting spike_detector to E, with label E_spike_detector and id (101,)
[__init__.py:129 - INFO]   - Attaching spike_detector with gid [(102,)] to population I
[__init__.py:438 - INFO] - Connecting spike_detector to I, with label I_spike_detector and id (102,)
[__init__.py:134 - INFO] Randomizing initial values:
[__init__.py:95 - INFO] - Randomizing V_m state in Population E
[__init__.py:95 - INFO] - Randomizing V_m state in Population I
[__init__.py:941 - INFO] ========================================================
[__init__.py:942 - INFO]  Brunel BRN architecture (NEST-simulated):
[__in


Loading default neuron parameter - iaf_psc_delta, fixed voltage threshold, fixed absolute refractory time


* Create an input sequencer and sequence. 

In [4]:
alphabet_size = 5
T = 100

sequencer = SymbolicSequencer(label='random', set_size=alphabet_size)

[sequences.py:132 - INFO] Generating symbolic sequencer


* Create a [vector embedding](embeddings.ipynb#section3.1) (or any other) for this example

In [5]:
emb = VectorEmbeddings(vocabulary=sequencer.tokens).binary_codeword(dim=100, density=0.2)

[embeddings.py:28 - INFO] Creating symbolic embeddings
[embeddings.py:91 - INFO] - 10 symbols encoded as binary-codeword


* Generate discrete stimulus sequence and convert it to continuous input by unfolding the stimulus representations 
in time (see [unfolding embeddings](embeddings.ipynb#section4.1.2)), or, alternatively, convert each token to a unique 
spatiotemporal spike pattern (see [frozen noise embeddings](embeddings.ipynb#section4.1.1)):

In [6]:
signal_pars = {'duration': 50.,'amplitude': 150.,'kernel': ('box', {}),'dt': resolution}
signal = emb.unfold(to_signal=True, **signal_pars)

[embeddings.py:28 - INFO] Creating symbolic embeddings
[embeddings.py:362 - INFO] Populating Stimulus Set: 
[embeddings.py:91 - INFO] - 10 symbols encoded as continuous signal unfolding


* Encode stimuli (see [encodings](encodings.ipynb#section6)), the input sequence will be provided later:

In [7]:
encoder = NESTEncoder('inhomogeneous_poisson_generator', label='poisson-input', dim=emb.embedding_dimensions)

[__init__.py:35 - INFO] Creating Generators: 
[__init__.py:38 - INFO] - poisson-input [(103,)-(202,)]


* Connect encoders

In [8]:
# input synapses
input_synapses = {
    'connect_populations': [('E', encoder.name), ('I', encoder.name), ],
    'weight_matrix': [None, None],
    'conn_specs': [{'rule': 'all_to_all'}, {'rule': 'all_to_all'}],
    'syn_specs': [{'model': 'static_synapse', 'delay': 0.1, 'weight': 3.},
                  {'model': 'static_synapse', 'delay': 0.1, 'weight': 3.}]
}
in_to_snn_connections = InputMapper(source=encoder, target=snn, parameters=input_synapses)

[__init__.py:116 - INFO] Connecting input generators: poisson-input -> Brunel BRN
[__init__.py:127 - INFO]     - ('E', 'poisson-input') [static_synapse]
[__init__.py:127 - INFO]     - ('I', 'poisson-input') [static_synapse]


* Create and connect [state extractors](state_sampling.ipynb): 
    * For this example, we will use the simplest sampling strategy whereby the population response vectors are gathered at stimulus offset

In [9]:
extractor_parameters = {
        'E_Vm_@offset': {
            'population': 'E',
            'variable': 'V_m',
            'sampling_times': ['stim_offset']}}

----

# **12. Decoding and Readout - massively parallel information processing**

The decoding process consists of combining the input-driven population responses $\mathbf{x}(t) \in \mathbb{R}_{\mathrm{N}}$ to approximate the desired target outputs $\mathbf{z}(t) = f(\mathbf{u}(t)) \in \mathbb{R}_{\mathrm{N_{out}}}$:

$$\mathbf{y}(t) = g (\mathbf{W}^{\mathrm{out}} \mathbf{x}(t) + \mathbf{b}^{\mathrm{out}})$$

where $\mathbf{y}(t)$ is an estimate of $\mathbf{z}(t)$ obtained by (linearly) combining population responses with coefficients $\mathbf{W}^{\mathrm{out}}$. Finding $\mathbf{W_{\mathrm{out}}}$ amounts to minimizing the error between target output and estimated output:

$$E_{\theta} = \langle \mathcal{L}(\mathbf{y}, \mathbf{z}) + R_{\theta}\rangle_{T}$$

to find the decoder parameters $\theta=\{\mathbf{W}^{\mathrm{out}} \in \mathbb{R}_{N\times N_{\mathrm{out}}}, \mathbf{b}^{\mathrm{out}} \in \mathbb{R}_{N}\}$. 

To achieve this, we collect the population states $X \in \mathbb{R}_{\mathrm{N\times T}}$ and the target outputs $Z \in \mathbb{R}_{\mathrm{N_{out}\times T}}$ for a given data batch of length $T$. During a training phase, we calibrate a supervised (often linear) readout to establish the proper mappings.

**Note:** Often the mappings are time-discrete (symbolic), whereas the states are continous. In these cases, we need to adequately sample and observe the continuous dynamics at discrete times ($t^{*}$, see [state extraction](state_sampling.ipynb))

## 12.1. The `Decoder` class and `Readout` algorithms

There are several algorithms available. In this example, we will employ a battery of readouts for a single `StateMatrix` on a simple classification task and compare their properties. For this purpose, we specify the parameters of the `Decoder`(s) to connect to the main `SpikingNetwork`. The decoding parameters have the following format: 

```python
decoding_parameters = {
    'readout_label': {
        'algorithm': 'pinv', # readout algorithm
        'extractor': extractor_label # state extractor label
        'save': True # store intermediate weights after each training epoch
    }
}
```

All readouts are instantiated via scikit-learn.

### 12.1.1. Single update readouts

```python
'pinv', 'ridge', 'logistic', 'perceptron', 'svm-linear', 'svm-rbf' 
```

Some algorithms do not support incremental updates, i.e. **each new batch will overwrite previous results**. These are retained, mostly for legacy reasons, but can only be correctly used if only one training batch is given. Otherwise, each batch is processed independently and the training outcome will refer only to the last batch as all previous results will be overwritten. 

If operating on a single batch, there are simple straightforward solutions. These algorithms include:

* Moore-Penrose direct pseudo-inverse (`pinv`) - quadratic loss function, no regularization

* Ridge regression (`ridge`) - quadratic loss function, L2 weight regularization

* Logistic regression (`logistic`) - classifier, log-loss, no regularization

* Linear SVM (`svm-linear`) - classifier, Huber (soft-margin) loss, with or without regu

* Non-linear SVM (`svm-rbf`) - classifier, with radial basis function kernel


### 12.1.2. Incremental readouts

```python
'force', 'sgd-reg', 'sgd-class', 'ridge-sgd', 'pinv-sgd', 'logistic-sgd', 'svm-linear-sgd'
```

When processing multiple batches of data, readout learning needs to be done incrementally following each batch. To achieve this, we make use of estimators that support `partial_fit` in `sklearn` or use recursive methods (the `FORCE` algorithm implements local recursive least squares). 



The options currently implemented are:

* **FORCE learning (`force`)**: is more effective if the output can drive the system, as modifications are instantaneous. In the absence of output feedback, this method tends to overfit the training data. The output weights are updated as:



---


* **Stochastic Gradient Descent (SGD)**: approximates the error gradient by considering a single training example at a time and incrementally updating the decoder weights $w$ and biases $b$:

$\Delta w = -\eta\left(\alpha \frac{\partial R_{w}}{\partial w}+\frac{\partial \mathcal{L}\left(w^{T} x_{i}+b, y_{i}\right)}{\partial w}\right)$

$\Delta b = -\eta\left(\alpha \frac{\partial \mathcal{L}\left(w^{T} x_{i}+b, y_{i}\right)}{\partial b}\right)$
    
$\eta$ is the learning rate, which can either be a constant hyperparameter or adaptive (setting the `learning_rate` parameter to `"optimal"`, `"invscaling"` or `"adaptive"` (see [sklearn documentation](https://scikit-learn.org/stable/modules/sgd.html) for more details). By default, and to minimize the number of free parameters, we use the `"optimal"` learning rate, whereby 

$$\eta(t)=\frac{1}{\alpha\left(t_{0}+t\right)}$$


Depending on the choice of loss function and regularization type, we can implement:
   - Least-squares regression (`"pinv-sgd"`) - quadratic loss, no regularization ($\alpha=0$), somewhat similar to recursive least squares
   - Ridge regression (`"ridge-sgd"`) - quadratic loss, L2 regularization ($\alpha$ is a free hyper-parameter)
   - Logistic regression (`"logistic-sgd"`) - Log-loss, with or without L2 regularization ($\alpha$ is a free hyper-parameter) 
   - Support Vector Classification (`"svm-linear-sgd"`) - Hinge (soft-margin) loss, with or without L2 regularization ($\alpha$ is a free hyper-parameter)
   
The above are special cases which are separated in the current implementation, for convenience. However, the `sklearn` implementation of SGD can accomodate a variety of different algorithms for either regression or classification problems, by specifying different combinations of the types of `loss` and `penalty`, as well as the hyperparameter values for $\alpha$ and $\eta_{0}$. By setting the decoder to use `"sgd-reg"` or `"sgd-class"`, the system runs a large search over these combinations to find the best decoder possible (note: for large datasets and large state-spaces, this may be prohibitively costly). 


**Notes:** There are some peculiarities of using `partial_fit` that may skew the results. The user is advised to pay attention to these:
* All hyperparameters are chosen using CV on the first training batch. So, make sure it is representative and large enough.
* This is particularly important when training classifiers ("sgd-class", "logistic-sgd" and "svm-linear-sgd"), as they may be unable to cope with unseen examples. To circumvent this, one may need to pass the `classes` argument when calling `partial_fit`
* `sgd-reg` and `sgd-class` encompass all other `SGD` objects, which means the hyperparameter search will be long and costly...
* Feature scaling and regularization ...


## 12.2. The `OuputMapper` class

All trained readout's coefficients / weights ($\mathbf{W}^{\mathrm{out}}$) are held in a separate class which stores the weights to memory, and also contains some analysis and plotting routines.

In [3]:
from fna.decoders.output_mapper import OutputMapper

[func for func in dir(OutputMapper) if callable(getattr(OutputMapper, func)) and not func.startswith("__")]

['get_connections',
 'measure_stability',
 'plot_connections',
 'plot_distributions',
 'plot_spectral_radius',
 'plot_weights',
 'save',
 'set_connections',
 'update_weights']

---

Let's compare the algorithms:

In [10]:
decoding_parameters = {
    'readout_E_ridge': {'algorithm': 'ridge', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_pinv': {'algorithm': 'pinv', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_logistic': {'algorithm': 'logistic', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_perceptron': {'algorithm': 'perceptron', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_svm-linear': {'algorithm': 'svm-linear', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_svm-rbf': {'algorithm': 'svm-rbf', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_force': {'algorithm': 'force', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_reg': {'algorithm': 'sgd-reg', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_class': {'algorithm': 'sgd-class', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_ridge-sgd': {'algorithm': 'ridge-sgd', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_pinv-sgd': {'algorithm': 'pinv-sgd', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_log-sgd': {'algorithm': 'logistic-sgd', 'extractor': 'E_Vm_@offset', 'save': True},
    'readout_E_svm-sgd': {'algorithm': 'svm-linear-sgd', 'extractor': 'E_Vm_@offset', 'save': True}
}

In [11]:
n_epochs = 2
n_batches = 10
batch_size = T
continuous = True
stim_onset = 0.1

**Batch processing:**
1. Parse an initial batch and discard it (inputs and responses) - discard initial transients

In [13]:
# Transient set
batch_sequence = sequencer.generate_random_sequence(T=10)
batch_stim_seq = signal.draw_stimulus_sequence(batch_sequence, onset_time=stim_onset, continuous=continuous, intervals=None)
snn.process_batch(batch_label='transient', encoder=encoder, stim_seq=batch_stim_seq)
stim_onset = snn.next_onset_time()

[sequences.py:161 - INFO] Generating a random sequence of length 10, from a set of 10 symbols
[embeddings.py:655 - INFO] Generating stimulus sequence: 10 symbols
[embeddings.py:605 - INFO] Concatenating stimulus sequence
[__init__.py:695 - INFO] 
[__init__.py:696 - INFO] ################## Simulating batch transient


2. Training phase - Training proceeds in batches, with the readouts tuned after each batch. After training, a validation is run on the same batch to quantify the evolution of the readout tuning process and how it might reflect the underlying dynamics (particularly relevant in systems that change with time).

**Note:** in ANNs, training typically also involves adjusting the internal connection weights as requires a single target (see [learning]()). 

In [15]:
# generate training batches
batch_labels = ['train_batch={}'.format(batch+1) for batch in range(n_batches)]
batch_seq = [sequencer.generate_random_sequence(T=batch_size) for _ in range(n_batches)]
batch_targets = [sequencer.generate_default_outputs(batch_sequence, max_memory=0, max_prediction=0,
                    max_chunk=0, chunk_memory=False, chunk_prediction=False) for batch_sequence in batch_seq]

[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols


In [16]:
# create and connect extractors and decoders
snn.connect_state_extractors(extractor_parameters, encoder=encoder, input_mapper=in_to_snn_connections,
                             stim_duration=signal_pars['duration'], stim_isi=None, stim_onset=stim_onset,
                             to_memory=True)
snn.create_decoder(decoding_parameters)
total_delay = in_to_snn_connections.total_delay + snn.state_extractor.max_extractor_delay

[extractors.py:490 - INFO] Connecting extractors (SpikingExtractor):
[extractors.py:496 - INFO] 	- State [E_Vm_@offset] from populations E [V_m]
[extractors.py:309 - INFO] 	  - Extractor (offset sampling)
[extractors.py:310 - INFO] 		- offset = 550.2 ms (stimulus onset + total delay + sim_res step)
[extractors.py:311 - INFO] 		- interval = 50.0 ms
[extractors.py:529 - INFO] 		- total delays = 0.2 ms (0.1 encoder + 0.1 extractor delay)
[extractors.py:530 - INFO] 		- NEST device id(s): [203]


In [18]:
# train 
for epoch in range(n_epochs):
    for batch in range(n_batches):
        batch_label = '{}_epoch={}'.format(batch_labels[batch], epoch+1)
        batch_input = signal.draw_stimulus_sequence(batch_seq[batch], onset_time=stim_onset,continuous=continuous, intervals=None)

        snn.train(batch_label, n_batches*n_epochs, stim_seq=batch_input, encoder=encoder, target_outputs=batch_targets[batch], total_delay=total_delay)

        stim_onset = snn.next_onset_time()

[embeddings.py:655 - INFO] Generating stimulus sequence: 100 symbols
[embeddings.py:605 - INFO] Concatenating stimulus sequence
[__init__.py:79 - INFO] Initializing and connecting decoder...
[__init__.py:86 - INFO] Creating readouts:
[readouts.py:48 - INFO]   - readout_E_ridge trained with ridge on classification, using state E_Vm_@offset
[readouts.py:48 - INFO]   - readout_E_pinv trained with pinv on classification, using state E_Vm_@offset
[readouts.py:48 - INFO]   - readout_E_logistic trained with logistic on classification, using state E_Vm_@offset
[readouts.py:48 - INFO]   - readout_E_perceptron trained with perceptron on classification, using state E_Vm_@offset
[readouts.py:48 - INFO]   - readout_E_svm-linear trained with svm-linear on classification, using state E_Vm_@offset
[readouts.py:48 - INFO]   - readout_E_svm-rbf trained with svm-rbf on classification, using state E_Vm_@offset
[readouts.py:48 - INFO]   - readout_E_elast trained with elastic on classification, using state 

3. Test phase - To evaluate the readouts' performance, we process a test batch and measure its ability to predict the target using only the system states

In [20]:
test_size = 100
test_label = 'fna_test_set'
test_sequence = sequencer.generate_random_sequence(T=test_size)
target_outputs = sequencer.generate_default_outputs(test_sequence, max_memory=0, max_chunk=0, max_prediction=0,
                                                    chunk_memory=False, chunk_prediction=False)
test_stim_seq = signal.draw_stimulus_sequence(test_sequence, onset_time=stim_onset,
                                                       continuous=continuous, intervals=None)

snn.predict(test_label, test_stim_seq, encoder, target_outputs, total_delay)
snn.decoder_accuracy(output_parsing='k-WTA', symbolic_task=True)

[sequences.py:161 - INFO] Generating a random sequence of length 100, from a set of 10 symbols
[embeddings.py:655 - INFO] Generating stimulus sequence: 100 symbols
[embeddings.py:605 - INFO] Concatenating stimulus sequence
[__init__.py:825 - INFO] Simulating test set fna_test_set
[extractors.py:592 - INFO] Deleting activity data from all state extractors
[extractors.py:540 - INFO] Reinitializing extractor [E_Vm_@offset]
[extractors.py:543 - INFO]   - Stopping recording from device [203]
[extractors.py:309 - INFO] 	  - Extractor (offset sampling)
[extractors.py:310 - INFO] 		- offset = 100550.5 ms (stimulus onset + total delay + sim_res step)
[extractors.py:311 - INFO] 		- interval = 50.0 ms
[__init__.py:695 - INFO] 
[__init__.py:696 - INFO] ################## Simulating batch fna_test_set
[extractors.py:805 - INFO] Extracting and storing recorded activity from state extractor E_Vm_@offset
[extractors.py:842 - INFO]   - Reading extractor 204 [V_m] at sampling offset
[extractors.py:584 -

IndexError: index 10 is out of bounds for axis 1 with size 10

In [21]:
snn.decoder_accuracy(output_parsing=None, symbolic_task=True)

[readouts.py:305 - INFO] Evaluating readout_E_ridge performance...


ValueError: Found input variables with inconsistent numbers of samples: [100, 10]